In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import time
import numpy as np
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import sklearn.metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
import time

model=load_model("sam.h5")
model = tf.keras.models.Model(inputs=[model.input], outputs=[model.output])
model.summary()

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.)
test_dir = r'F:/sampletest'
test_generator = test_datagen.flow_from_directory(directory =test_dir,
                                                 # batch_size = n_test,
                                                batch_size = 1,
                                                  target_size = (300, 300),
                                                  #class_mode = "categorical",
                                                 #shuffle = False
                                                 )

# number of layers in model 
NL=len(model.layers)

# Number of iterations
runs=50
p = np.zeros((runs,NL))


def time_per_layer(model, number_of_runs):
    new_model = model
    run= number_of_runs
    times = np.zeros((len(model.layers), 2))
    inp = np.ones((300, 300, 3))
    print("Layers","\t","Run time(ms)","\n")
    
    for j in range(0,run):
        print("Run is:","\t",j+1)
        for i in range(0, len(model.layers)):
            new_model = tf.keras.models.Model(inputs=[model.input], outputs=[model.layers[-(i+1)].output])

           # new_model.summary()
            new_model.predict(test_generator)

            t_s = time.time()
            new_model.predict(test_generator)
            t_e2 = time.time() - t_s
          #  print(len(new_model.layers), "\t", t_e2, "\n", )
            times[i, 1] = t_e2
            p[j][i]=t_e2
            del new_model


    return times

times = time_per_layer(model,runs)
ttl= np.zeros(NL)
for i in range(0,NL):
    tmp=0
    for j in range(0, runs):
        tmp=tmp+p[j][i]
    
    ttl[i]=tmp/runs   
    
ttl

x = [model.layers[-i].name for i in range(1,len(model.layers))]

print("Layer","\t", "Execution time(ms)")

for i in range(0,len(x)):
    #var= (ttl[i]-ttl[i+1])*1000
    print(x[i], "\t", (ttl[i]-ttl[i+1])*1000,"\n")

